In [1]:
import geopandas as gpd 
import pandas as pd
from shapely import ops
import os
import shapely
import numpy as np
import psycopg2
import pandana as pdna
import networkx as nx
import multiprocessing as mp
import matplotlib.pyplot as plt
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *

from shapely.geometry import *
shapely.speedups.enable()

%matplotlib inline 

In [2]:
con = psycopg2.connect(database="walkability", user="postgres", password=1234,
    host="172.19.118.205",port="5432")

In [3]:
# load data from pg database
con = psycopg2.connect(database="walkability", user="postgres", password=1234,
    host="172.19.118.205",port="5432")
pednet= gpd.read_postgis('SELECT * FROM public.pednet100m',con,crs={'init': 'epsg:2019'})
census = gpd.read_postgis('SELECT * FROM compliance.cssc_2016',con,crs={'init': 'epsg:2019'})

neighborhood = gpd.read_postgis('SELECT * FROM compliance.neighborhooda',con,crs={'init': 'epsg:2019'})


In [27]:
census.head(3)

,id,geom,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,...,a215,a216,a217,a218,a219,a220,a221,a222,a223,a224
0,1,(POLYGON ((320588.9306980086 4835855.647789237...,1,62,5350001.00,0001.00,35,Ontario,535,35535,...,0.0,535.0,325.0,210.0,150.0,55.0,55.0,35.0,20.0,0.0
1,2,"(POLYGON ((316642.831678931 4832470.554474635,...",2,63,5350002.00,0002.00,35,Ontario,535,35535,...,0.0,635.0,495.0,140.0,120.0,25.0,20.0,10.0,0.0,10.0
2,3,(POLYGON ((314351.8668838083 4845056.100542194...,3,87,5350264.00,0264.00,35,Ontario,535,35535,...,15.0,1280.0,905.0,375.0,280.0,90.0,50.0,45.0,10.0,40.0


In [47]:
from shapely.geometry import Point, MultiPoint, shape, mapping, Polygon, LineString
import geopandas as gpd
join = gpd.sjoin(pednet, census,  op='intersects')



In [52]:
join.ctname

,id_left,geom,gid_left,x_gid,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,...,a216,a217,a218,a219,a220,a221,a222,a223,a224,length
112258,77875,(LINESTRING (318222.2661333333 4841779.2157666...,51351,70146.0,19202.0,21965.0,9610.0,-6932.0,6932.0,-19203.0,...,3805.0,1925.0,1875.0,920.0,950.0,130.0,60.0,75.0,820.0,74.421857
112259,77876,(LINESTRING (318292.8375666667 4841802.8438333...,51351,70146.0,19202.0,21965.0,9610.0,-6932.0,6932.0,-19203.0,...,3805.0,1925.0,1875.0,920.0,950.0,130.0,60.0,75.0,820.0,74.421857
112260,77877,(LINESTRING (317915.4285599232 4841627.7036224...,51352,70148.0,19208.0,9673.0,21966.0,19203.0,19203.0,82705.0,...,3805.0,1925.0,1875.0,920.0,950.0,130.0,60.0,75.0,820.0,95.031179
112261,77878,(LINESTRING (318004.6266299616 4841660.4851612...,51352,70148.0,19208.0,9673.0,21966.0,19203.0,19203.0,82705.0,...,3805.0,1925.0,1875.0,920.0,950.0,130.0,60.0,75.0,820.0,95.031179
112262,77879,"(LINESTRING (318093.8247 4841693.2667, 318151....",51353,70147.0,19203.0,21966.0,21965.0,19202.0,19202.0,-19208.0,...,3805.0,1925.0,1875.0,920.0,950.0,130.0,60.0,75.0,820.0,85.046128


In [51]:
join["length"] = join["geom"].length


In [64]:
ljoin =join.groupby('ctname').sum()
ljoin.head(5)

,id_left,gid_left,x_gid,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,abs_next_r,...,a216,a217,a218,a219,a220,a221,a222,a223,a224,length
ctname,,,,,,,,,,,,,,,,,,,,,
0001.00,50394084,37574857,21918928.0,25176389.0,16848055.0,17690002.0,7924193.0,25005579.0,-7026736.0,24600466.0,...,338655.0,205725.0,132930.0,94950.0,34815.0,34815.0,22155.0,12660.0,0.0,42963.471473
0002.00,32485620,21292697,10662600.0,18376970.0,14900005.0,14599973.0,7174352.0,18152350.0,-8120676.0,18409044.0,...,268605.0,209385.0,59220.0,50760.0,10575.0,8460.0,4230.0,0.0,4230.0,25168.767279
0003.00,22175403,16120135,14244132.0,13908606.0,8479386.0,8381636.0,5017118.0,13728398.0,-5827371.0,13080297.0,...,120930.0,34750.0,86180.0,48650.0,37530.0,22240.0,22240.0,2780.0,15290.0,15595.553223
0004.00,5523971,4586705,4199542.0,3678292.0,2605013.0,2481061.0,1375756.0,3346106.0,-1424254.0,3547584.0,...,679840.0,336000.0,343840.0,211680.0,132160.0,70560.0,48160.0,21840.0,61600.0,6713.947570
0005.00,6693000,6088811,4204613.0,4093676.0,2725432.0,2580136.0,1292383.0,4096397.0,-1171897.0,4051189.0,...,854830.0,400040.0,455520.0,276670.0,178850.0,129940.0,94170.0,35770.0,48910.0,7832.599830


In [70]:
lcount = join.groupby('ctname').count()
lcount = lcount.rename(columns={'length': 'count'})
lcount.head(5)                  

,id_left,geom,gid_left,x_gid,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,...,a216,a217,a218,a219,a220,a221,a222,a223,a224,count
ctname,,,,,,,,,,,,,,,,,,,,,
0001.00,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633
0002.00,423,423,423,423,423,423,423,423,423,423,...,423,423,423,423,423,423,423,423,423,423
0003.00,278,278,278,278,278,278,278,278,278,278,...,278,278,278,278,278,278,278,278,278,278
0004.00,112,112,112,112,112,112,112,112,112,112,...,112,112,112,112,112,112,112,112,112,112
0005.00,146,146,146,146,146,146,146,146,146,146,...,146,146,146,146,146,146,146,146,146,146


In [98]:
newdf = lcount.loc[:, ['count']]
newdf.head(3)

,count
ctname,
0001.00,633
0002.00,423
0003.00,278


In [99]:
dfs = ljoin.merge(newdf, on='ctname', how='left')
dfs.head(5)

,id_left,gid_left,x_gid,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,abs_next_r,...,a217,a218,a219,a220,a221,a222,a223,a224,length,count
ctname,,,,,,,,,,,,,,,,,,,,,
0001.00,50394084,37574857,21918928.0,25176389.0,16848055.0,17690002.0,7924193.0,25005579.0,-7026736.0,24600466.0,...,205725.0,132930.0,94950.0,34815.0,34815.0,22155.0,12660.0,0.0,42963.471473,633
0002.00,32485620,21292697,10662600.0,18376970.0,14900005.0,14599973.0,7174352.0,18152350.0,-8120676.0,18409044.0,...,209385.0,59220.0,50760.0,10575.0,8460.0,4230.0,0.0,4230.0,25168.767279,423
0003.00,22175403,16120135,14244132.0,13908606.0,8479386.0,8381636.0,5017118.0,13728398.0,-5827371.0,13080297.0,...,34750.0,86180.0,48650.0,37530.0,22240.0,22240.0,2780.0,15290.0,15595.553223,278
0004.00,5523971,4586705,4199542.0,3678292.0,2605013.0,2481061.0,1375756.0,3346106.0,-1424254.0,3547584.0,...,336000.0,343840.0,211680.0,132160.0,70560.0,48160.0,21840.0,61600.0,6713.947570,112
0005.00,6693000,6088811,4204613.0,4093676.0,2725432.0,2580136.0,1292383.0,4096397.0,-1171897.0,4051189.0,...,400040.0,455520.0,276670.0,178850.0,129940.0,94170.0,35770.0,48910.0,7832.599830,146


In [ ]:
%matplotlib inline
import pysal as ps
import pandas as pd
import numpy as np
from pysal.contrib.viz import mapping as maps

In [ ]:
import matplotlib.pyplot as plt

import geopandas as gpd

tx = gpd.read_file(neighborhood)
hr10 = ps.Quantiles(data.HR90, k=10)
f, ax = plt.subplots(1, figsize=(9, 9))
tx.assign(cl=hr10.yb).plot(column='walkscoren', categorical=True, \
        k=10, cmap='OrRd', linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.title("HR90 Deciles")
plt.show()

In [ ]:
#This script works in ArcMap


import arcpy

#Change these filepaths to fit your data
polygons = "C:\\Data\\Data.mdb\\Polygon"
lines = "C:\\Data\\Data.mdb\\Line"
workspace = "C:\\Data\\Data.mdb"

arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

#Adds fields to the input polygon feature
arcpy.AddField_management(polygons, "PolyOID", "TEXT", "", "", "", "Polygon OID", "NULLABLE", "")
arcpy.AddField_management(polygons, "TotalLength", "TEXT", "", "", "", "Total Length", "NULLABLE", "")

#Creates temporary feature layers
arcpy.MakeFeatureLayer_management(polygons, "Temp_poly")
arcpy.MakeFeatureLayer_management(lines, "Temp_line")

#Collects individual polygons by OBJECTID
poly_OID = []
poly_cursor = arcpy.SearchCursor(polygons)
for row in poly_cursor:
    poly_OID.append(row.getValue("OBJECTID"))
del row
del poly_cursor

#Cycles through each polygon by OBJECTID and selects all lines COMPLETELY WITHIN the polygon
#Records the total shape length of all the lines
TotalLength = []
for polygon in poly_OID:
    expression = """{0} = {1}""".format("[OBJECTID]", polygon)
    arcpy.SelectLayerByAttribute_management("Temp_poly", "NEW_SELECTION", expression)
    arcpy.SelectLayerByLocation_management("Temp_line", "COMPLETELY_WITHIN", "Temp_poly")

    combinedLength = 0
    line_cursor = arcpy.SearchCursor("Temp_line")
    for row in line_cursor:
        combinedLength = combinedLength + row.getValue("SHAPE_Length")
    TotalLength.append(combinedLength)
    del row
    del line_cursor

#Adds the shape length total and PolyOID to each polygon
count = 0
cursor = arcpy.UpdateCursor(polygons)
for row in cursor:
    row.setValue("PolyOID", poly_OID[count])
    row.setValue("TotalLength", TotalLength[count])
    cursor.updateRow(row)
    count = count + 1
del row
del cursor